In [ ]:
using TensorOperations, TensorKit
import TensorKit.×
using FLoops
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")
include("./models/Hubbard_Z2SU2.jl")
# include("./models/tJ_Z2.jl")
@show pkgversion(TensorKit)
using JLD2, HDF5
using Strided
Strided.enable_threads()

In [ ]:
para = Dict{Symbol,Any}()
para[:t] = 1.0
para[:U] = 8.0
para[:μ] = 4.0
para[:pspace] = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
Hij = get_op_Hubbard("hij", para)
@show Hij

Hijp = permute(Hij, ((2, 1), (4, 3)))
@show Hijp
@assert Hij == Hijp
nothing

In [ ]:
# @show Threads.nthreads(:default)
# @show Threads.nthreads(:interactive)
# @show Threads.nthreadpools()

@sync begin
    Threads.@spawn for xx in 1:3
        @show (xx, Threads.threadid())
        @floop for pp in 1:2
            println("xx$xx, pp$pp, thread$(Threads.threadid())")
            sleep(3)
        end
        sleep(5)
    end
    Threads.@spawn for yy in 1:3
        @show (yy, Threads.threadid())
        @floop for pp in 1:2
            println("yy$yy, pp$pp, thread$(Threads.threadid())")
            sleep(3)
        end
        sleep(5)
    end
    Threads.@spawn for zz in 1:3
        @show (zz, Threads.threadid())
        @floop for pp in 1:2
            println("zz$zz, pp$pp, thread$(Threads.threadid())")
            sleep(3)
        end
        sleep(5)
    end
    Threads.@spawn for kk in 1:3
        @show (kk, Threads.threadid())
        @floop for pp in 1:2
            println("kk$kk, pp$pp, thread$(Threads.threadid())")
            sleep(3)
        end
        sleep(5)
    end
end

nothing


In [ ]:
# @sync begin
#     Threads.@spawn begin
        @floop for xx in 1:3
            # 左右
            println("xx$xx, thread$(Threads.threadid())")
            sleep(3)
        end
        @floop for yy in 1:3
            # 上下
            println("yy$yy, thread$(Threads.threadid())")
            sleep(3)
        end
#     end
# end

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./models/tJ_Z2.jl")
include("./simple_update_Fermionic/simple_update.jl")
include("./fast_full_update_Fermionic/fast_full_update.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")

ipeps, envs, para = load("/home/tcmp2/JuliaProjects/tJZ2_Lx2Ly2_SU_t3.0t'0.51J1.0J'0.0289h0.6mu5.2_ipepsEnv_D6chi140.jld2", "ipeps", "envs", "para")
Lx = ipeps.Lx
Ly = ipeps.Ly
@show para[:χ]
ipepsbar = bar(ipeps)

Obs2si = Cal_Obs_2site(ipeps, ipepsbar, envs, ["Δₛ", "Δₛdag"], para; site1=[1, 1], site2=[0, 2], get_op=get_op_tJ)

nothing

In [ ]:
pspace = Rep[ℤ₂](0 => 1, 1 => 2)
include("./models/tJ_Z2.jl")
OpZ = Z(pspace)

Fdag₊, F₊ = FdagF₊(pspace)
@tensor fdagf₊[p1, p3; p2, p4] := OpZ[p1, p1in] * Fdag₊[p1in, p2, a] * F₊[a, p3, p4]
@show Fdag₊
Fdag₋, F₋ = FdagF₋(pspace)
@show Fdag₋
@tensor fdagf₋[p1, p3; p2, p4] := Fdag₋[p1, p2, a] * F₋[a, p3in, p4] * OpZ[p3, p3in]

F₊, Fdag₊ = FFdag₊(pspace)
@tensor ffdag₊[p1, p3; p2, p4] := OpZ[p1, p1in] * F₊[p1in, p2, a] * Fdag₊[a, p3, p4]
F₋, Fdag₋ = FFdag₋(pspace)
@tensor ffdag₋[p1, p3; p2, p4] := F₋[p1, p2, a] * Fdag₋[a, p3in, p4] * OpZ[p3, p3in]

@show ishermitian(fdagf₊ - ffdag₊ + fdagf₋ - ffdag₋)

In [ ]:
include("./models/Hubbard_Z2SU2.jl")
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)

OpZ = Z(pspace)
Opnd = nd(pspace)
Opn = n(pspace)
OpI = id(pspace)
Fdag, F = FdagF(pspace)
@tensor fdagf[p1, p3; p2, p4] := Fdag[p1in, p2, a] * F[a, p3, p4]
# @tensor fdagf[p1, p3; p2, p4] := Fdag[p1, p2, a] * F[a, p3, p4]

F, Fdag = FFdag(pspace)
@tensor ffdag[p1, p3; p2, p4] := F[p1in, p2, a] * Fdag[a, p3, p4]

@show ishermitian(fdagf - ffdag)

In [ ]:
ipeps, envs, para = load("/home/tcmp2/JuliaProjects/tJZ2_Lx2Ly2_SU_t3.0t'0.51J1.0J'0.0289h0.6mu5.2_ipepsEnv_D6chi140.jld2", "ipeps", "envs", "para")


In [ ]:
using ITensors
using ITensorMPS

sites = siteinds("Electrons", 2; conserve_qns=true)
os = OpSum()
    os += -t, "Cdagup", ii, "Cup", ii + 1
    os += -t, "Cdagdn", ii, "Cdn", ii + 1
    os += -t, "Cdagup", ii + 1, "Cup", ii
    os += -t, "Cdagdn", ii + 1, "Cdn", ii
    os += J, "Sz", ii, "Sz", ii + 1
    os += J / 2, "S+", ii, "S-", ii + 1
    os += J / 2, "S-", ii + 1, "S+", ii
    os += -J / 4, "Ntot", ii, "Ntot", ii + 1
